In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier #Second is for data size >10k. CLASSIFICATION.
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor #as above. REGRESSION.
# import lightgbm

In [ ]:
df = pd.read_csv('train_top_20k.csv')
df = df[df.columns.tolist()[1:]] #Drops the duplicated index values in the first column.
df.dropna(subset=['target'], inplace=True)
df.head()

#### Some basic notes on the column definitions, units, etc:
- imbalance_size is defined as the "amount unmatched at the current reference price (in USD)"
- imbalance_buy_sell_flag gives the DIRECTION of the auction imbalance
- reference_price is the price at which paired shares are maximized, the imbalance is minimized and the distance from the bid-ask midpoint is minimized (in that order). **What does this mean exactly?**

In [ ]:
weight_arr = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]
stock_id_list = np.arange(200)
weights_df = pd.DataFrame({'stock_id':stock_id_list, 'weight':weight_arr})
df = df.merge(weights_df, how='left', on='stock_id')
df.head()

In [ ]:
#Cell to explore the above loaded df if needed without disturbing what is below.

# df[~df['far_price'].isna()]

# test = df.groupby(['stock_id']).rolling(3)['bid_price'].mean().fillna(method='bfill')
# df['ewmas_3'] = np.zeros(len(df))
# df['ewmas_3'] = np.where(df['stock_id']==0, test, df['ewmas_3'])
# print(df.groupby('stock_id').rolling(1)['bid_price'].mean()[0].head(20))
# test.head(20)

In [ ]:
# df[df['stock_id']==0].head(20)

#### Next cell will be adding some BASIC features to the data as manual (domain-specific) augmentation. 
TBD: how to incorporate formal kernel methods if that is of interest.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#Add basics like categorical target (for classification tree), book skew + other simple data transforms.
df['book_skew'] = np.log(df['bid_size']/df['ask_size'])
df['target_cat'] = np.where(df['target'] < 0, -1, np.where(df['target'] > 0, 1, 0)) #should we make this binary?

#Add some moving-average style transformations of important variables.
ewma_windows = [3,5,10]
ewma_cols = ['imbalance_size', 'book_skew', 'bid_price', 'ask_price']
for w in ewma_windows:
    for col in ewma_cols:
        df[f'ewmas_{col}_{w}'] = np.zeros(len(df))
        for id in df['stock_id'].unique():
            ma = df.groupby(['stock_id']).rolling(w)[col].mean().fillna(method='bfill').loc[id] #This is a little sketchy, maybe worth changing?
            # df[f'ewmas_{col}_{w}'] = np.where(df['stock_id']==id, ma, df[f'ewmas_{col}_{w}'])
            df[f'ewmas_{col}_{w}'][ma.index] = ma
        # print(f'ewmas_{col}_{w}', np.sum(df[f'ewmas_{col}_{w}']==0))



# df[['book_skew', 'ewmas_book_skew_5', 'bid_price', 'ask_price', 'ewmas_bid_price_5', 'ewmas_ask_price_5', "stock_id", 'date_id']].head(20)

In [ ]:
# test = df[df['ewmas_imbalance_size_3']== 0][['stock_id', 'date_id', 'imbalance_size', 'ewmas_imbalance_size_3']]
# test['stock_id'].unique()

In [ ]:
#Derive the train/test params for our toy model.
print(df.time_id.unique()) #gonna use up to 40 for training and the rest for testing

In [ ]:
#Define the train-test split.
X_train = df[df['time_id'] <=40].drop(columns=['target', 'target_cat'])
X_train = X_train.fillna(1e-9) #For imputing NaN values. Does this value make a big difference to performance?
X_test = df[df['time_id'] >40].drop(columns=['target', 'target_cat'])
X_test = X_test.fillna(1e-9) #See above.
print("Xtrain length", len(X_train)); print("Xtest length", len(X_test))
y_train = df[df['time_id'] <=40]['target']
y_test = df[df['time_id'] >40]['target']

#Filter for final feature set.
features = [c for c in df.columns if not c in ['stock_id','date_id','seconds_in_bucket','target_cat', 'target']]
print("Number of features used: ", len(features))
X_train = X_train[features]
X_test = X_test[features]

X_train.head()

In [ ]:
#Fit classifier.
interesting_cols = ['seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap', 'time_id', 'weight', 'book_skew']
clf = GradientBoostingRegressor(loss='absolute_error', learning_rate=0.05, n_estimators=1000, verbose=1, random_state=42)
clf.fit(X_train[interesting_cols], y_train)

In [ ]:
#Run prediction.
y_pred = clf.predict(X_test[interesting_cols])
prediction_mae = np.linalg.norm(y_pred-y_test,ord=1)/len(y_pred)
print(prediction_mae)

In [ ]:
#Fits a classifier with early stopping and min impurity criterion for limiting overfitting.
#Should we include cost-complexity pruning as well?
clf2 = GradientBoostingRegressor(loss='absolute_error', learning_rate=0.05, n_estimators=1000, verbose=1, random_state=42,
                                validation_fraction=0.2, n_iter_no_change=3, min_impurity_decrease=0.1)
clf2.fit(X_train, y_train)

In [ ]:
#Prediction using the limited classifier.
y_pred2 = clf2.predict(X_test)
prediction2_mae = np.linalg.norm(y_pred2-y_test,ord=1)/len(y_pred2)
print(prediction2_mae)

In [ ]:
plt.bar(clf.feature_names_in_, height=clf.feature_importances_)
plt.figure(figsize=(50,100))

In [ ]:
from pydotplus import graph_from_dot_data
from IPython.display import Image
from sklearn.tree import export_graphviz

In [ ]:
dot_data = export_graphviz(
    clf2.estimators_[1,0],
    out_file=None, filled=True, rounded=True,
    special_characters=True,
    proportion=True, impurity=False, # enable them if you want
    feature_names=clf.feature_names_in_
)

Get the 20th tree

In [ ]:
graph = graph_from_dot_data(dot_data)
png = graph.create_png()

In [ ]:
from pathlib import Path
Path('./tree_20.png').write_bytes(png)
# Display
Image(png)

In [ ]:
X_test.iloc[1]

In [ ]:
clf2.estimators_[1,0].predict(X_test)[1]